In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu


In [6]:
class MaskedConv2D(nn.Conv2d):
	def __init__(self,mask_type, *args, **kwargs):
		super(MaskedConv2D, self).__init__()
		self.mask_type = mask_type
		assert mask_type in ['A', 'B'], "Unknown Mask Type"
		self.register_buffer('mask', self.weight.data.clone())
		_, depth, height, width = self.weight.size()
		self.mask.fill_(1)
		if mask_type =='A':
			self.mask[:,:,height//2,width//2:] = 0
			self.mask[:,:,height//2+1:,:] = 0
		else:
			self.mask[:,:,height//2,width//2+1:] = 0
			self.mask[:,:,height//2+1:,:] = 0

	def forward(self, x):
		self.weight.data*=self.mask
		return super(MaskedConv2D, self).forward(x)

In [9]:
class PixelCNN(nn.Module):
	def __init__(self, nb_layer_block=8, channels=64, device=None):
		super(PixelCNN, self).__init__()
		self.nb_block = nb_layer_block
		self.channels = channels
		self.layers = {}
		self.device = device
		#first convolution (cf TABLE 1)
		self.ConvA = nn.Sequential(
			MaskedConv2D('A',3, 2*channels, kernel_size=7, bias=False),
			nn.ReLU(True)
		)
		#Residual blocks for PixelCNN (figure 5)
		self.multiple_blocks = nn.Sequential(
			nn.Conv2d(2*channels,channels, kernel_size = 1, bias=False),
			nn.ReLU(True),
			MaskedConv2D('B',channels,channels, kernel_size = 3, bias=False),
			nn.ReLU(True),
			nn.Conv2d(channels,2*channels, kernel_size = 1, bias=False),
			nn.ReLU(True)
		)
		self.end = nn.Sequential(
			nn.ReLU(True),
			MaskedConv2D('B',2*channels, 2 * channels, kernel_size = 1, bias=False),
			nn.ReLU(True),
			MaskedConv2D('B',2* channels,2 * channels, kernel_size = 1, bias=False),
			nn.Conv2d( 2 * channels, 256, 1)
		)

	def residual_block(self, x):
		return (x + self.multiple_blocks(x))
	
	def forward(self,x):
		x = self.ConvA(x)
		for i in range(self.nb_block):
			x = residual_block(x)
		x = self.end(x)

SAVING FITTED MODEL

In [ ]:
PATH = './model_1_Alex.pth'
torch.save(net.state_dict(), PATH)